In [1]:
pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 21.9 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Fo

In [21]:
!wget --no-check-certificate \
  https://github.com/24wemy/dataS/archive/refs/heads/main.zip \
  -O /tmp/test_train.zip

--2024-06-07 09:47:13--  https://github.com/24wemy/dataS/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/24wemy/dataS/zip/refs/heads/main [following]
--2024-06-07 09:47:13--  https://codeload.github.com/24wemy/dataS/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.116.9
Connecting to codeload.github.com (codeload.github.com)|140.82.116.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49567082 (47M) [application/zip]
Saving to: ‘/tmp/test_train.zip’

/tmp/test_train.zip 100%[===================>]  47.27M  63.7MB/s    in 0.7s    

2024-06-07 09:47:14 (63.7 MB/s) - ‘/tmp/test_train.zip’ saved [49567082/49567082]



In [22]:
!unzip /tmp/test_train.zip -d /tmp && rm /tmp/test_train.zip


Archive:  /tmp/test_train.zip
a045da324f854f2702c3c243121bebdc6168b71c
replace /tmp/dataS-main/dataset/boat/N_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /tmp/dataS-main/dataset/boat/N_1.jpg  
  inflating: /tmp/dataS-main/dataset/boat/N_10.jpg  
  inflating: /tmp/dataS-main/dataset/boat/N_100.jpg  
  inflating: /tmp/dataS-main/dataset/boat/N_101.jpg  
  inflating: /tmp/dataS-main/dataset/boat/N_102.jpg  
  inflating: /tmp/dataS-main/dataset/boat/N_103.jpg  
  inflating: /tmp/dataS-main/dataset/boat/N_104.jpg  
  inflating: /tmp/dataS-main/dataset/boat/N_105.jpg  
  inflating: /tmp/dataS-main/dataset/boat/N_106.jpg  
  inflating: /tmp/dataS-main/dataset/boat/N_107.jpg  
  inflating: /tmp/dataS-main/dataset/boat/N_108.jpg  
  inflating: /tmp/dataS-main/dataset/boat/N_109.jpg  
  inflating: /tmp/dataS-main/dataset/boat/N_11.jpg  
  inflating: /tmp/dataS-main/dataset/boat/N_110.jpg  
  inflating: /tmp/dataS-main/dataset/boat/N_111.jpg  
  inflating: /tmp/dataS-main/dataset/

In [23]:
!pip install split_folders

In [5]:
import os
import numpy as np
import cv2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


In [24]:
base_dir = "/tmp/dataS-main/dataset"
output_dir = "/tmp/split_dataset"

In [25]:
def load_images_from_directory(directory, width, height):
    image_data = []
    image_target = []
    classes = os.listdir(directory)
    for title in classes:
        class_dir = os.path.join(directory, title)
        if os.path.isdir(class_dir):
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    if img.shape[-1] == 1:
                        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                    else:
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (width, height))
                    image_data.append(img)
                    image_target.append(title)
    return np.array(image_data), np.array(image_target)


In [8]:
all_images, all_labels = load_images_from_directory(base_dir, 224, 224)

lb = LabelBinarizer()
all_labels = lb.fit_transform(all_labels)

train_images, validation_images, train_labels, validation_labels = train_test_split(
    all_images, all_labels, test_size=0.2, random_state=42, stratify=all_labels)

train_images = train_images.astype('float32') / 255.0
validation_images = validation_images.astype('float32') / 255.0


In [9]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.BatchNormalization())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(len(lb.classes_), activation='softmax'))

for layer in base_model.layers:
    layer.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ModelCheckpoint('YogaPoseModel.keras', save_best_only=True),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=1e-4)
]


history = model.fit(
    train_images, train_labels,
    epochs=100,
    batch_size=32,
    validation_data=(validation_images, validation_labels),
    callbacks=callbacks
)

Epoch 1/100
75/75 ━━━━━━━━━━━━━━━━━━━━ 32s 309ms/step - accuracy: 0.2436 - loss: 2.0339 - val_accuracy: 0.5317 - val_loss: 1.2120 - learning_rate: 1.0000e-04
Epoch 2/100
75/75 ━━━━━━━━━━━━━━━━━━━━ 18s 52ms/step - accuracy: 0.4987 - loss: 1.3029 - val_accuracy: 0.7033 - val_loss: 0.8931 - learning_rate: 1.0000e-04
Epoch 3/100
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - accuracy: 0.6112 - loss: 1.0027 - val_accuracy: 0.7583 - val_loss: 0.7326 - learning_rate: 1.0000e-04
Epoch 4/100
75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.6959 - loss: 0.8283 - val_accuracy: 0.7850 - val_loss: 0.6392 - learning_rate: 1.0000e-04
Epoch 5/100
75/75 ━━━━━━━━━━━━━━━━━━━━ 7s 70ms/step - accuracy: 0.7330 - loss: 0.7453 - val_accuracy: 0.8133 - val_loss: 0.5784 - learning_rate: 1.0000e-04
Epoch 6/100
75/75 ━━━━━━━━━━━━━━━━━━━━ 9s 48ms/step - accuracy: 0.7624 - loss: 0.6547 - val_accuracy: 0.8267 - val_loss: 0.5350 - learning_rate: 1.0000e-04
Epoch 7/100
75/75 ━━━━━━━━━━━━━━━━━━━━ 7s 71ms/step - accurac